# Experiment07: 连续优化问题的EA算法实现

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ConstantKernel as C

In [2]:
np.random.seed(0)

BOUNDS=np.array([[0.0, 2.0]])##搜索区间[0,2]
POP_SIZE=40##种群规模
GENS=40##迭代次数
NOISE_STD=0.08##噪声标准差
N_INTIAL=15##初始样本数量
MUT_SCALE=0.05##变异尺度

In [3]:
def true_function(x):
    return np.sin(2*np.pi*x)

def noisy_function(x):
    return true_function(x) + np.random.normal(0, NOISE_STD)

In [4]:
x_train=np.random.uniform(0,2,size=(N_INTIAL,1))##均匀分布生成初始样本
y_train=np.array([noisy_function(x) for x in x_train])

kernel=C(1.0)*RBF(0.5)+WhiteKernel(NOISE_STD**2)##定义核函数，包含一个常数核、一个RBF核和一个白噪声核
gp=GaussianProcessRegressor(kernel=kernel,normalize_y=True)##创建高斯过程回归模型
gp.fit(x_train, y_train)

,kernel,1**2 * RBF(le..._level=0.0064)
,alpha,1e-10
,optimizer,'fmin_l_bfgs_b'
,n_restarts_optimizer,0
,normalize_y,True
,copy_X_train,True
,n_targets,None
,random_state,None
,kernel__k1,1**2 * RBF(length_scale=0.5)
,kernel__k2,WhiteKernel(n..._level=0.0064)
,kernel__k1__k1,1**2


In [5]:
def tournament_selection(pop,fitness):
    new_pop=[]
    for _ in range(len(pop)):
        idx=np.random.choice(len(pop),3)
        ##随机选择三个个体进行锦标赛选择,比较它们的适应度，选择适应度最低的个体作为父代
        best=idx[np.argmin(fitness[idx])]
        ##选择适应度最低的个体作为父代
        new_pop.append(pop[best])
    return np.array(new_pop)

In [6]:
def mutate(pop):
    pop=pop+np.random.normal(0,MUT_SCALE,size=pop.shape)##对种群进行变异，添加高斯噪声
    pop=np.clip(pop,0,2)##将变异后的个体限制在搜索区间内
    return pop

In [7]:
def run_EA(mode="true"):
    pop=np.random.uniform(0,2,size=(POP_SIZE,1))##初始化种群

    for gen in range(GENS):
        if mode=="true":
            fitness=np.array([true_function(x) for x in pop])##计算适应度，使用真实函数值
        else:
            fitness=np.array([gp.predict(x.reshape(1,-1))[0] for x in pop])##计算适应度，使用高斯过程预测值

        pop=tournament_selection(pop,fitness)##选择
        pop=mutate(pop)##变异

    real_values=np.array([true_function(x) for x in pop])##计算最终种群的真实函数值
    best_idx=np.argmin(real_values)##找到适应度最低的个体索引
    return pop[best_idx][0], real_values[best_idx]##返回最佳个体的输入值和对应的真实函数值

In [8]:
x_true, val_true = run_EA("true")
x_surr, val_surr = run_EA("surrogate")

print("EA with real fitness:", val_true)
print("EA with surrogate fitness:", val_surr)


EA with real fitness: [-0.99954318]
EA with surrogate fitness: [-0.99909153]
